In [11]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [5]:
df = pd.read_excel('Actividad_Retail_Clientes.xlsx')

In [ ]:
print(df.head())

   ID  HoraIngreso  Compra  MontoCompra
0   1           13       1         6.99
1   2           19       0         0.00
2   3           16       0         0.00
3   4           15       1        26.24
4   5           12       0         0.00


In [6]:
df.shape

(450, 4)

1. Estimar la proporcion de clientes que compran utilizando la muestra sistematica.

In [7]:
N = len(df)
compradores = df['Compra'].sum()
proporcion_muestra = compradores / N

print(f"Clientes en la muestra: {N}")
print(f"Clientes que compraron: {compradores}")
print(f"Proporción de compradores en la muestra: {proporcion_muestra:.4f} ({proporcion_muestra*100:.2f}%)")

Clientes en la muestra: 450
Clientes que compraron: 167
Proporción de compradores en la muestra: 0.3711 (37.11%)


2. Calcular el promedio y la desviacion estandar del monto de compra.

In [8]:
monto_promedio_total = df['MontoCompra'].mean()
monto_std_total = df['MontoCompra'].std(ddof=1)

print(f"Promedio del monto (todos los clientes): ${monto_promedio_total:.2f}")
print(f"Desviación estándar (todos los clientes): ${monto_std_total:.2f}")

Promedio del monto (todos los clientes): $11.00
Desviación estándar (todos los clientes): $22.99


In [9]:
df_compradores = df[df['Compra'] == 1]
monto_promedio_compradores = df_compradores['MontoCompra'].mean()
monto_std_compradores = df_compradores['MontoCompra'].std(ddof=1)

print(f"\nPromedio del monto (solo compradores): ${monto_promedio_compradores:.2f}")
print(f"Desviación estándar (solo compradores): ${monto_std_compradores:.2f}")


Promedio del monto (solo compradores): $29.63
Desviación estándar (solo compradores): $29.56


3. Construir un intervalo de confianza al 95% para la proporcion de compradores y para el monto promedio.

In [15]:
alpha = 0.05
z_alpha_2 = st.norm.ppf(1 - alpha/2)

# Error estándar para la proporción
se_proporcion = np.sqrt(proporcion_muestra * (1 - proporcion_muestra) / N)
margen_error_prop = z_alpha_2 * se_proporcion

ic_prop_inferior = max(0, proporcion_muestra - margen_error_prop)
ic_prop_superior = min(1, proporcion_muestra + margen_error_prop)

print(f"Intervalo de confianza para la proporción de compradores:")
print(f"  [{ic_prop_inferior:.4f}, {ic_prop_superior:.4f}]")
print(f"  [{ic_prop_inferior*100:.2f}%, {ic_prop_superior*100:.2f}%]")

# Intervalo de confianza para el promedio del monto
se_monto = monto_std_total / np.sqrt(N)
margen_error_monto = z_alpha_2 * se_monto

ic_monto_inferior = monto_promedio_total - margen_error_monto
ic_monto_superior = monto_promedio_total + margen_error_monto

print(f"\nIntervalo de confianza para el monto promedio:")
print(f"  [${ic_monto_inferior:.2f}, ${ic_monto_superior:.2f}]")

Intervalo de confianza para la proporción de compradores:
  [0.3265, 0.4157]
  [32.65%, 41.57%]

Intervalo de confianza para el monto promedio:
  [$8.87, $13.12]


4. Estimar el numero total de compradores entre los 450 observados.

In [16]:
poblacion_total = 450
compradores_estimados = poblacion_total * proporcion_muestra
se_compradores = poblacion_total * se_proporcion

ic_compradores_inferior = max(0, compradores_estimados - z_alpha_2 * se_compradores)
ic_compradores_superior = min(poblacion_total, compradores_estimados + z_alpha_2 * se_compradores)

print(f"Número estimado de compradores: {compradores_estimados:.0f}")
print(f"Intervalo de confianza: [{ic_compradores_inferior:.0f}, {ic_compradores_superior:.0f}]")

Número estimado de compradores: 167
Intervalo de confianza: [147, 187]


5. Analizar si existen horas pico o valle en los datos observados.

In [17]:
hora_stats = df.groupby('HoraIngreso').agg({
    'ID': 'count',
    'Compra': ['sum', 'mean'],
    'MontoCompra': ['mean', 'sum']
}).round(2)

hora_stats.columns = ['Clientes', 'Compradores', 'Tasa_Compra', 'Monto_Promedio', 'Monto_Total']

print("Estadísticas por hora de ingreso:")
print(hora_stats)

# Identificar horas pico y valle
horas_con_mas_clientes = hora_stats.nlargest(3, 'Clientes')
horas_con_menos_clientes = hora_stats.nsmallest(3, 'Clientes')

print(f"\nHORAS PICO (más clientes observados):")
for hora in horas_con_mas_clientes.index:
    clientes = horas_con_mas_clientes.loc[hora, 'Clientes']
    print(f"  {hora}:00 - {clientes} clientes")

print(f"\nHORAS VALLE (menos clientes observados):")
for hora in horas_con_menos_clientes.index:
    clientes = horas_con_menos_clientes.loc[hora, 'Clientes']
    print(f"  {hora}:00 - {clientes} clientes")

# Análisis de comportamiento de compra por hora
print(f"\nHORAS CON MAYOR TASA DE COMPRA:")
horas_alta_compra = hora_stats.nlargest(3, 'Tasa_Compra')
for hora in horas_alta_compra.index:
    tasa = horas_alta_compra.loc[hora, 'Tasa_Compra']
    print(f"  {hora}:00 - {tasa*100:.1f}% de tasa de compra")

print(f"\nHORAS CON MAYOR MONTO PROMEDIO:")
horas_alto_monto = hora_stats.nlargest(3, 'Monto_Promedio')
for hora in horas_alto_monto.index:
    monto = horas_alto_monto.loc[hora, 'Monto_Promedio']
    if monto > 0:
        print(f"  {hora}:00 - ${monto:.2f} promedio")

Estadísticas por hora de ingreso:
             Clientes  Compradores  Tasa_Compra  Monto_Promedio  Monto_Total
HoraIngreso                                                                 
9                  27            7         0.26            7.31       197.44
10                 31           11         0.35            9.11       282.49
11                 21            7         0.33           12.28       257.92
12                 39           12         0.31            8.77       341.86
13                 72           27         0.38           11.75       846.31
14                 57           21         0.37           10.83       617.21
15                 64           32         0.50           16.36      1047.27
16                 46           15         0.33            7.17       329.64
17                 48           22         0.46           13.24       635.69
18                 23            7         0.30            9.97       229.28
19                 16            5        